input: dataframe with a column named "book_url"
do: scrape every row
return: dataframe with "rank" and original price of the url but without the book name

In [4]:
import sys
import time
import random
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re
from selenium import webdriver

In [13]:
def set_up_driver(headless=True):
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
    return webdriver.Chrome(service=Service(r"C:\Users\USER\OneDrive\桌面\C2L2\chromedriver.exe"), options=chrome_options)

In [14]:
def crawler(url):
    driver = set_up_driver()
    driver.get(url)
    driver.refresh()

    i = 0
    while True:

        time.sleep(1)
        driver.refresh()

        # 隨機選取秒數
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        i = i + 1
        print(i, end=" ")
        refuse_request = [
            "Request was throttled. Please wait a moment and refresh the page",
            "      503 - Service Unavailable Error",
            "Amazon.comEnter the characters you see belowSorry, we just need to make sure you're not a robot. For best results, please make sure your browser is accepting cookies.Type the characters you see in this image:Try different imageContinue shoppingConditions of UsePrivacy Policy          © 1996-2014, Amazon.com, Inc. or its affiliates          ",
        ]
        if soup.text.replace("\n", "") in refuse_request:
            driver.close()
            time.sleep(1 * i)

            driver = set_up_driver()
            driver.get(url)
        else:
            break

    driver.close()
    return soup

# 爬內容

In [10]:
df = pd.read_csv("sampled_book_data.csv")

In [ ]:
df = pd.DataFrame()

In [11]:
all_rank, other_info, origin_price = [], [], []

In [18]:
ind = 0
while ind < 2:
    soup = crawler(f"https://www.amazon.com/s?k={df['title'][ind]}")

    if soup.text.replace("\n", "").strip(" ") == "Page Not Found":
        all_rank.append("_")
        other_info.append("_")
        origin_price.append("_")
        print("[", ind, "]", "Page Not Found!!!!!!!")
        ind = ind + 1
        continue

    try:
        ul = soup.find_all("ul", class_="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list")
        rank_str = ul[1].find("span", class_="a-list-item").text
        rank_str = rank_str.split(" in Books (")[0].strip("  Best Sellers Rank:  ")
        all_rank.append(rank_str)

        detail = soup.find("ul", class_="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list").text
        detail = detail.replace(
            "\n                                    \u200f\n                                        :\n                                    \u200e\n                                 ", ":"
        )
        other_info.append(detail)

    except IndexError:
        try:
            ul = soup.find("table", class_="a-keyvalue a-vertical-stripes a-span6")
            rank_str = ul.find_all("tr")[-1].find_all("span")[1].text.split(" in", 1)[0]
            all_rank.append(rank_str)

            detail = ul.text
            other_info.append(detail)

        except AttributeError:
            try:
                ul = soup.find_all("table", class_="a-keyvalue prodDetTable")[1]
                rank_str = ul.find_all("tr")[-2].find_all("span")[1].text.split(" in", 1)[0]
                all_rank.append(rank_str)

                detail = ul.text
                other_info.append(detail)

            except IndexError:
                ind = ind
                continue

    except AttributeError:
        try:
            all_rank.append("_")
            other_info.append("_")
            origin_price.append("_")
            print("[", ind, "]", "detail only has consumer review???????")
            ind = ind + 1
            continue
        except AttributeError:
            ind = ind
            continue

    try:
        p = soup.find("div", class_="a-section a-spacing-none a-text-right")
        p = p.find("span", class_="a-color-secondary a-text-strike")
        origin_price.append(p.text)
    except AttributeError:
        origin_price.append(df.price[ind])

    print("[", ind, "]", df.title[ind])
    ind = ind + 1

1 1 1 1 1 1 1 1 1 1 1 1 1 1 

Exception ignored in: <function Service.__del__ at 0x000001AB4CDFECB0>
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 185, in __del__
    self.stop()
  File "c:\Users\USER\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\USER\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 131, in send_remote_shutdown_command
    if not self.is_connectable():
  File "c:\Users\USER\anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 120, in is_connectable
    return utils.is_connectable(self.port)
  File "c:\Users\USER\anaconda3\lib\site-packages\selenium\webdriver\common\utils.py", line 101, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "c:\Users\USER\anaconda3\lib\socket.py", line 833, in create_connection
    sock.connect(sa)
KeyboardInterrupt: 


In [ ]:
Best_Seller_other_info = pd.DataFrame(
    {'all_rank':all_rank,
     'origin_price':origin_price,
     'other_info':other_info})
Best_Seller_other_info.to_csv('Best_Seller_other_info_3500_3749.csv',index=False)